In [1]:
import sys
sys.path.append("../") 

In [2]:
import pandas as pd
import numpy as np

import scanpy as sc

import seaborn as sns
import matplotlib.pyplot as plt
import torch

from models import SDCI
from utils import create_intervention_dataset

In [3]:
adata = sc.read("/home/justinhong/dcdfg_preprocess/perturb-cite-seq/SCP1064/ready/control/gene_filtered_adata.h5ad")
adata

AnnData object with n_obs × n_vars = 57523 × 1657
    obs: 'library_preparation_protocol', 'condition', 'MOI', 'sgRNA', 'UMI_count', 'sgRNAs', 'n_genes', 'targets', 'regimes'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'targeted'
    uns: 'hvg', 'log1p'
    layers: 'counts'

In [4]:
X_df = pd.DataFrame(adata.X.todense(), index=adata.obs_names, columns=adata.var_names)
X_df["perturbation_label"] = adata.obs["targets"]
dataset = create_intervention_dataset(X_df)

In [5]:
def run_sdci(): 
    wandb.init()
    mv_flavor = wandb.config.mv_flavor
    s1_alpha = wandb.config.s1_alpha
    s2_alpha = wandb.config.s2_alpha
    s1_beta = wandb.config.s1_beta
    s2_beta = wandb.config.s2_beta
    max_gamma = wandb.config.max_gamma

    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using {device}")
    model = SDCI(model_variance_flavor=mv_flavor)
    model.train(
        train_dataset,
        device=device,
        log_wandb=True,
        verbose=False,
        stage1_kwargs={"n_epochs": 100, "alpha": s1_alpha, "beta": s1_beta, "n_epochs_check": 5},
        stage2_kwargs={"n_epochs": 100, "alpha": s2_alpha, "beta": s2_beta, "max_gamma": max_gamma, "n_epochs_check": 5}
    )
    val_rec_loss = model._model.reconstruction_loss(val_dataset[:][0].to(device), mask_interventions_oh=val_dataset[:][1].to(device)).cpu().detach().item()

    min_dag_threshold = model.compute_min_dag_threshold()
    n_edges_min_dag = model.get_adjacency_matrix().sum()
    
    wandb.log(dict(val_rec_loss=val_rec_loss, min_dag_threshold=min_dag_threshold, n_edges_min_dag=n_edges_min_dag))
    wandb.finish()

In [6]:
sweep_configuration = {
    "method" : "bayes",
    "name": "sdci_new_sim_sweep",
    "metric": {
        "goal": "minimize",
        "name": "val_rec_loss",
    },
    "parameters": {
        "s1_alpha": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s2_alpha": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s1_beta": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s2_beta": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "max_gamma": {"max": 1000, "min": 100, "distribution": "log_uniform_values"},
        "mv_flavor": {"values": ["nn"]},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 11,
    }
}

In [7]:
import wandb
wandb.login()

sweep_id = wandb.sweep(sweep=sweep_configuration, project="SDCI_perturb_cite_seq")
wandb.agent(sweep_id, function=run_sdci)

wandb: Currently logged in as: justinhong (azizi-causal-perturb). Use `wandb login --relogin` to force relogin


Create sweep with ID: c55u19ta
Sweep URL: https://wandb.ai/azizi-causal-perturb/SDCI_perturb_cite_seq/sweeps/c55u19ta


wandb: Agent Starting Run: efid05rs with config:
wandb: 	max_gamma: 379.8147342065447
wandb: 	mv_flavor: nn
wandb: 	s1_alpha: 0.01710698860588679
wandb: 	s1_beta: 9.746829410362016e-05
wandb: 	s2_alpha: 0.0001275049488058736
wandb: 	s2_beta: 8.220218668276032e-05


Using cpu


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: v0evv4rl with config:
wandb: 	max_gamma: 788.5012912260496
wandb: 	mv_flavor: nn
wandb: 	s1_alpha: 0.0001521174327874298
wandb: 	s1_beta: 1.9629115357268193e-05
wandb: 	s2_alpha: 4.915065785584007e-05
wandb: 	s2_beta: 0.0058967113188155545


Using cpu


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 0: loss=2562.72, gamma=0.00
Epoch 5: loss=1551.51, gamma=0.00
Epoch 10: loss=1447.75, gamma=0.00
Epoch 15: loss=1385.96, gamma=0.00
Epoch 20: loss=1337.06, gamma=0.00
Epoch 25: loss=1303.06, gamma=0.00
Epoch 30: loss=1271.99, gamma=0.00
Epoch 35: loss=1254.63, gamma=0.00
Epoch 40: loss=1236.33, gamma=0.00
Epoch 45: loss=1222.99, gamma=0.00
Epoch 50: loss=1207.96, gamma=0.00
Epoch 55: loss=1204.39, gamma=0.00
Epoch 60: loss=1191.77, gamma=0.00
Epoch 65: loss=1190.78, gamma=0.00
Epoch 70: loss=1182.83, gamma=0.00
Epoch 75: loss=1185.46, gamma=0.00
Epoch 80: loss=1170.36, gamma=0.00
Epoch 85: loss=1168.90, gamma=0.00
Epoch 90: loss=1160.68, gamma=0.00
Epoch 95: loss=1157.12, gamma=0.00
Fraction of possible edges in mask: 0.4162181691833151
Epoch 0: loss=2241.35, gamma=0.00
Epoch 5: loss=1866.27, gamma=39.82
Epoch 10: loss=1893.37, gamma=79.65


wandb: Ctrl + C detected. Stopping sweep.


## Final Run

In [7]:
mv_flavor = "nn"
s1_alpha = 3e-4
s2_alpha = 2e-4
s1_beta = 1e-3
s2_beta = 1e-2
max_gamma = 150

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")
model = SDCI(model_variance_flavor=mv_flavor)
model.train(
    train_dataset,
    device=device,
    log_wandb=True,
    verbose=False,
    stage1_kwargs={"n_epochs": 100, "alpha": s1_alpha, "beta": s1_beta, "n_epochs_check": 5},
    stage2_kwargs={"n_epochs": 200, "alpha": s2_alpha, "beta": s2_beta, "max_gamma": max_gamma, "n_epochs_check": 5}
)
val_rec_loss = model._model.reconstruction_loss(val_dataset[:][0].to(device), mask_interventions_oh=val_dataset[:][1].to(device)).cpu().detach().item()

min_dag_threshold = model.compute_min_dag_threshold()
n_edges_min_dag = model.get_adjacency_matrix().sum()


Using cpu


wandb: Currently logged in as: justinhong (azizi-causal-perturb). Use `wandb login --relogin` to force relogin


Epoch 0: loss=2568.90, gamma=0.00
Epoch 5: loss=1589.86, gamma=0.00
Epoch 10: loss=1518.75, gamma=0.00
Epoch 15: loss=1484.07, gamma=0.00
Epoch 20: loss=1456.98, gamma=0.00
Epoch 25: loss=1440.65, gamma=0.00
Epoch 30: loss=1425.02, gamma=0.00
Epoch 35: loss=1412.95, gamma=0.00
Epoch 40: loss=1400.56, gamma=0.00
Epoch 45: loss=1393.00, gamma=0.00
Epoch 50: loss=1391.28, gamma=0.00
Epoch 55: loss=1388.51, gamma=0.00
Epoch 60: loss=1393.43, gamma=0.00
Epoch 65: loss=1388.32, gamma=0.00
Epoch 70: loss=1406.35, gamma=0.00
Epoch 75: loss=1396.85, gamma=0.00
Epoch 80: loss=1399.29, gamma=0.00
Epoch 85: loss=1401.64, gamma=0.00
Epoch 90: loss=1396.68, gamma=0.00
Epoch 95: loss=1416.35, gamma=0.00
Fraction of possible edges in mask: 0.10504401691549066
Epoch 0: loss=2104.48, gamma=0.00
Epoch 5: loss=1770.53, gamma=3.77
Epoch 10: loss=1775.59, gamma=7.54
Epoch 15: loss=1783.65, gamma=11.31
Epoch 20: loss=1774.82, gamma=15.08
Epoch 25: loss=1786.33, gamma=18.84
Epoch 30: loss=1782.88, gamma=22.61

In [9]:
dag_thresh_adj_mtx = model.get_adjacency_matrix()
no_thresh_adj_mtx = model.get_adjacency_matrix(threshold=False)
fixed_thresh_adj_mtx = (no_thresh_adj_mtx > 0.3).astype(int)

In [10]:
min_dag_threshold

0.5586318969726562

In [11]:
n_edges_min_dag

1574

In [13]:
np.savetxt("results/dag_thresh_adj_mtx.csv", dag_thresh_adj_mtx, delimiter=",")
np.savetxt("results/no_thresh_adj_mtx.csv", no_thresh_adj_mtx, delimiter=",")
np.savetxt("results/fixed_thresh_adj_mtx.csv", fixed_thresh_adj_mtx, delimiter=",")